In [1]:
import numpy as np
import json
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
data_point = '0200'
npy_file_path = f'/home/colino/hawk/synthetic_ground_truth/tests/SimulationAlignment_1/bounding_box_3d_{data_point}.npy'
labels_file_path = f'/home/colino/hawk/synthetic_ground_truth/tests/SimulationAlignment_1/bounding_box_3d_labels_{data_point}.json'
data_npy = np.load(npy_file_path, allow_pickle=True)

In [9]:
print("Contents of the .npy file:")
print(data_npy[:])

Contents of the .npy file:
[(0, -13.349487  , -1.1616669e+01,  4.8628118e-02, 13.349426  , 1.1616638e+01, 6.4486008e+01, [[ 9.3227820e-03,  3.6174215e-03,  0.0000000e+00,  0.0000000e+00], [-3.6174215e-03,  9.3227820e-03,  0.0000000e+00,  0.0000000e+00], [ 0.0000000e+00,  0.0000000e+00,  9.9999998e-03,  0.0000000e+00], [-7.2977443e+00,  2.7479644e+00, -5.9559842e-08,  1.0000000e+00]], 0.    )
 (0, -13.349433  , -1.1616821e+01,  4.8628118e-02, 13.349438  , 1.1616577e+01, 6.4486008e+01, [[ 3.4622647e-04, -9.9940049e-03,  0.0000000e+00,  0.0000000e+00], [ 9.9940049e-03,  3.4622647e-04,  0.0000000e+00,  0.0000000e+00], [ 0.0000000e+00,  0.0000000e+00,  9.9999998e-03,  0.0000000e+00], [-1.6340977e+01, -8.1233901e-01, -5.9559827e-08,  1.0000000e+00]], 0.    )
 (1,  -0.50000095, -5.0000143e-01, -5.0000000e-01,  0.49999952, 4.9999905e-01, 4.9999994e-01, [[ 9.1060328e-01, -4.1328162e-01,  0.0000000e+00,  0.0000000e+00], [ 8.2656324e-02,  1.8212065e-01,  0.0000000e+00,  0.0000000e+00], [ 0.000000

In [10]:
# Extract the first column
first_elements = [row[0] for row in data_npy]

# Count unique values in the first column
unique_values, counts = np.unique(first_elements, return_counts=True)

In [11]:
print(unique_values, counts)

[0 1] [2 5]


In [12]:
with open(labels_file_path, 'r') as labels_file:
    labels_data = json.load(labels_file)

In [13]:
print("Contents of the label file")
print(labels_data)

Contents of the label file
{'0': {'class': 'wetfloorsign'}, '1': {'class': 'alignmentobj'}}


In [14]:
data_npy_full = [list(row) for row in data_npy]

In [15]:
sign_data = [row for row in data_npy_full if row[0] in [1]]

In [16]:
geometries = []

In [11]:
sign_points = []
sign_transforms = []
for row in sign_data:
    x_min = row[1]
    y_min = row[2]
    x_max = row[3]
    y_max = row[4]
    z_min = row[5]
    z_max = row[6]
    
    bbox_points = [
        [x_min, y_min, z_min],
        [x_min, y_min, z_max],
        [x_min, y_max, z_min],
        [x_min, y_max, z_max],
        [x_max, y_min, z_min],
        [x_max, y_min, z_max],
        [x_max, y_max, z_min],
        [x_max, y_max, z_max]
    ]
    
    sign_points.append(bbox_points)
    
    transform = row[7]
    sign_transforms.append(transform.T)
    
    
sign_points = np.array(sign_points).astype(np.float64)
sign_transforms = np.array(sign_transforms).astype(np.float64)   
    
print(sign_points)
print(sign_transforms)

[[[-1.33494873e+01 -1.16166687e+01  1.16166382e+01]
  [-1.33494873e+01 -1.16166687e+01  6.44860077e+01]
  [-1.33494873e+01  1.33494263e+01  1.16166382e+01]
  [-1.33494873e+01  1.33494263e+01  6.44860077e+01]
  [ 4.86281179e-02 -1.16166687e+01  1.16166382e+01]
  [ 4.86281179e-02 -1.16166687e+01  6.44860077e+01]
  [ 4.86281179e-02  1.33494263e+01  1.16166382e+01]
  [ 4.86281179e-02  1.33494263e+01  6.44860077e+01]]

 [[-1.33494329e+01 -1.16168213e+01  1.16165771e+01]
  [-1.33494329e+01 -1.16168213e+01  6.44860077e+01]
  [-1.33494329e+01  1.33494377e+01  1.16165771e+01]
  [-1.33494329e+01  1.33494377e+01  6.44860077e+01]
  [ 4.86281179e-02 -1.16168213e+01  1.16165771e+01]
  [ 4.86281179e-02 -1.16168213e+01  6.44860077e+01]
  [ 4.86281179e-02  1.33494377e+01  1.16165771e+01]
  [ 4.86281179e-02  1.33494377e+01  6.44860077e+01]]]
[[[ 9.32278205e-03 -3.61742149e-03  0.00000000e+00 -7.29774427e+00]
  [ 3.61742149e-03  9.32278205e-03  0.00000000e+00  2.74796438e+00]
  [ 0.00000000e+00  0.000000

In [12]:
sign_pcds = []

for i, row in enumerate(sign_points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(row)
    pcd.transform(sign_transforms[i])
    sign_pcds.append(pcd)
    
geometries += sign_pcds

In [13]:
sign_bboxes = []

for i, pcd in enumerate(sign_pcds):
    sign_bbox = pcd.get_oriented_bounding_box()
    sign_bbox.color = [1,0,0]
    sign_bboxes.append(sign_bbox)
    
geometries += sign_bboxes

In [14]:
transporter_points = o3d.utility.Vector3dVector()
transporter_data = [row for row in data_npy_full if row[0] in [0]]

for row in transporter_data:
    x_min = row[1]
    y_min = row[2]
    x_max = row[3]
    y_max = row[4]
    z_min = row[5]
    z_max = row[6]
    
    points = [
        [x_min, y_min, z_min],
        [x_min, y_min, z_max],
        [x_min, y_max, z_min],
        [x_min, y_max, z_max],
        [x_max, y_min, z_min],
        [x_max, y_min, z_max],
        [x_max, y_max, z_min],
        [x_max, y_max, z_max]
    ]
    
    transform = row[7].T

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.array(points).astype(np.float64))
    pcd.transform(np.array(transform).astype(np.float64))
    transporter_points.extend(pcd.points)
    
transporter_pcd = o3d.geometry.PointCloud()
transporter_pcd.points = transporter_points

geometries += [transporter_pcd]

In [15]:
transporter_bbox = transporter_pcd.get_oriented_bounding_box()
transporter_bbox.color = [1,0,0]
geometries += [transporter_bbox]

In [16]:
origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
geometries += [origin_frame]
#geometries = sign_pcds

In [17]:
o3d.visualization.draw_geometries(geometries)